# 環境設定

* 本程式推薦使用 GPU，執行速度會快很多。

In [ ]:
# 下載照片檔（Image）
import os
Dataset_File = "Frieren.jpg"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [ ]:
# 安裝 EasyOCR 套件
!pip install easyocr

# 辨識照片上的字

| Language Code | Language            | Language Code | Language       |
|---------------|---------------------|---------------|----------------|
| 'ch_sim'      | Simplified Chinese  | 'en'          | English        |
| 'ch_tra'      | Traditional Chinese | 'fr'          | French         |
| 'de'          | German              | 'it'          | Italian        |
| 'es'          | Spanish             | 'pt'          | Portuguese     |
| 'ru'          | Russian             | 'ar'          | Arabic         |
| 'he'          | Hebrew              | 'hi'          | Hindi          |
| 'ja'          | Japanese            | 'ko'          | Korean         |
| 'th'          | Thai                | 'vi'          | Vietnamese     |
| 'zh_cn'       | Chinese (Simplified) | 'zh_tw'       | Chinese (Traditional) |

In [ ]:
# 辨識照片上的文字
import easyocr

# 指定辨識繁體中文與英文
reader = easyocr.Reader(['ch_tra', 'en'])

# 會傳回來一個 Python List，內含辨識到的每一組文字資訊
results = reader.readtext(Dataset_File)

In [ ]:
# 每一組文字資訊，又包含「文字座標」、「文字本身」、「信心度」
for items in results:
  print("座標：", items[0])
  print("文字：", items[1])
  print("信心度：", items[2])
  print("---------------------------------------------------")

座標： [[713, 55], [769, 55], [769, 95], [713, 95]]
文字： 十
信心度： 0.016972056057964702
---------------------------------------------------
座標： [[331, 37], [875, 37], [875, 247], [331, 247]]
文字： 葬*芙莉蓮
信心度： 0.060725962437658695
---------------------------------------------------
座標： [[390, 177], [766, 177], [766, 229], [390, 229]]
文字： R T RI
信心度： 0.01301050642022479
---------------------------------------------------
座標： [[1059, 721], [1099, 721], [1099, 737], [1059, 737]]
文字： TOH0
信心度： 0.4000180661678314
---------------------------------------------------
座標： [[1060, 738], [1120, 738], [1120, 746], [1060, 746]]
文字： 1 nTminioT
信心度： 0.025372240267466453
---------------------------------------------------
座標： [[74, 747], [239, 747], [239, 788], [74, 788]]
文字： MUSE木棉花代理
信心度： 0.5268352225144601
---------------------------------------------------
座標： [[769, 761], [1125, 761], [1125, 779], [769, 779]]
文字： CKanehito lamada Tsukasa Abe Shogakukan/ Frieren Project
信心度： 0.43206531509795604
-------------

# 影片逐字稿

In [ ]:
# 安裝 YouTube 下載套件，從 YouTube 和其他網站下載影片
! pip install yt-dlp

# 安裝聲音格式轉.WAV 套件（.WAV = 一種非壓縮音訊格式）
! pip install pydub

# 從 GitHub 安裝最新版本的 Whisper 套件，用於語音識別（約 1 分鐘）
! pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-kwx50xnj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-kwx50xnj
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 M

In [ ]:
import subprocess
from pydub import AudioSegment

def youtube_downloader(url, file_name='audio'):
  try:
    # 使用 yt-dlp 下載 YouTube 影片的音訊
    subprocess.run(['yt-dlp', '-x', '--audio-format', 'mp3', '-o', f'{file_name}.%(ext)s', url])

    # 將 MP3 轉換成 WAV 格式
    audio = AudioSegment.from_file(file_name + '.mp3')
    audio.export(file_name + '.wav', format = 'wav')

    # 輸出成功
    print(f'影片以下載&轉成{file_name}.wav')


  except Exception as e:
    print(f'發生錯誤：{e}')

In [ ]:
import whisper
# 定義一個使用 Whisper 進行轉錄的函數
def transcribe_with_whisper(wav_basename='audio', model='base'):
    # 加載指定的 Whisper 模型
    model = whisper.load_model(model)

    # 使用模型轉錄音訊檔案，預設名稱為 'audio.wav'
    result = model.transcribe(f'{wav_basename}.wav')

    # 返回轉錄的文本
    return result['text']

In [ ]:
# 定義要下載的 YouTube 影片 URL
url = 'https://www.youtube.com/watch?v=sIWemv8OhGk'
# 設定下載的音訊檔案基礎名稱
audio_basename = 'one_piece_comics'

# 使用自定義的 YouTube 下載器函數下載影片並命名檔案
youtube_downloader(url, file_name=audio_basename)

# 使用 Whisper 模型將下載的音訊轉錄成文本
text = transcribe_with_whisper(wav_basename=audio_basename, model='medium')

# 列印轉錄的文本
print(text)

影片以下載&轉成one_piece_comics.wav


100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 87.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
